In [12]:
import numpy as np
from json import load
import spacy
import nltk
import string
from nltk.corpus import stopwords
from string import punctuation
from spacy.matcher import PhraseMatcher
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, MultinomialNB, CategoricalNB, BernoulliNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, RocCurveDisplay, auc, recall_score, f1_score, precision_score
import joblib
nlp = spacy.load("ru_core_news_sm")


In [2]:
not_discriptors = list()
with open('../../Data/Words/word_pairs_not_disc.txt', 'r', encoding='utf-8') as txt:
    for i in txt:
        not_discriptors.append(i.replace('\n', ''))

with open('../../Data/Events/type_of_events.json', 'r', encoding='utf-8') as js:
    events = load(js)

with open('../../Data/Datasets/data.json', 'r', encoding='utf-8') as js:
    discriptors = load(js)


In [3]:
dict_of_disc = {'expession': [], 'is_discr': []}
for subject in discriptors.values():
    for d in sum(subject.values(), []):
        dict_of_disc['expession'].append(nlp(d).vector)
        dict_of_disc['is_discr'].append(True)

for subject in not_discriptors:
    dict_of_disc['expession'].append(nlp(subject).vector)
    dict_of_disc['is_discr'].append(False)



In [4]:
features = np.array(dict_of_disc['expession'])
labels = np.array(list(map(int, dict_of_disc['is_discr'])))


In [5]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, random_state=71)


# Гаусовское распространение

## Подбираем гиппер-параметры.

In [6]:
gaussian_naive_bayes_optimal_f1 = GridSearchCV(GaussianNB(),
                                                        {'var_smoothing': np.arange(0, 1, 1e-1),
                                                         },
                                                        scoring=f1_score).fit(X_train, y_train)
gaussian_naive_bayes_optimal_precission = GridSearchCV(GaussianNB(),
                                                        {'var_smoothing': np.arange(0, 1, 1e-1),
                                                         },
                                                        scoring=f1_score).fit(X_train, y_train)
gaussian_naive_bayes_optimal_recall = GridSearchCV(GaussianNB(),
                                                        {'var_smoothing': np.arange(0, 1, 1e-1),
                                                         },
                                                        scoring=f1_score).fit(X_train, y_train)


c:\Users\Mike\Desktop\ITS_project\ITS_project\lib\site-packages\sklearn\model_selection\_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Mike\Desktop\ITS_project\ITS_project\lib\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
TypeError: f1_score() takes 2 positional arguments but 3 were given

  warnings.warn(
c:\Users\Mike\Desktop\ITS_project\ITS_project\lib\site-packages\sklearn\model_selection\_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Mike\Desktop\ITS_project\ITS_project\lib\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
TypeError: f1_score() takes 2 

## Формула Байеса

In [7]:
y_predicted_by_gaussian_naive_bayes_f1 = gaussian_naive_bayes_optimal_f1.predict(X_test)
y_predicted_by_gaussian_naive_bayes_precission = gaussian_naive_bayes_optimal_precission.predict(X_test)
y_predicted_by_gaussian_naive_bayes_recall = gaussian_naive_bayes_optimal_recall.predict(X_test)

## Оценка качества 

In [8]:
print('Accuracy')
print(accuracy_score(y_test, y_predicted_by_gaussian_naive_bayes_f1))
print('Confusion matrix')
print(confusion_matrix(y_test, y_predicted_by_gaussian_naive_bayes_f1))
print('Precision, Recall, F\n', classification_report(
    y_test, y_predicted_by_gaussian_naive_bayes_f1))



Accuracy
0.8913857677902621
Confusion matrix
[[ 85  11]
 [ 18 153]]
Precision, Recall, F
               precision    recall  f1-score   support

           0       0.83      0.89      0.85        96
           1       0.93      0.89      0.91       171

    accuracy                           0.89       267
   macro avg       0.88      0.89      0.88       267
weighted avg       0.89      0.89      0.89       267



In [9]:
print('Accuracy')
print(accuracy_score(y_test, y_predicted_by_gaussian_naive_bayes_recall))
print('Confusion matrix')
print(confusion_matrix(y_test, y_predicted_by_gaussian_naive_bayes_recall))
print('Precision, Recall, F\n', classification_report(
    y_test, y_predicted_by_gaussian_naive_bayes_recall))



Accuracy
0.8913857677902621
Confusion matrix
[[ 85  11]
 [ 18 153]]
Precision, Recall, F
               precision    recall  f1-score   support

           0       0.83      0.89      0.85        96
           1       0.93      0.89      0.91       171

    accuracy                           0.89       267
   macro avg       0.88      0.89      0.88       267
weighted avg       0.89      0.89      0.89       267



In [10]:
print('Accuracy')
print(accuracy_score(y_test, y_predicted_by_gaussian_naive_bayes_precission))
print('Confusion matrix')
print(confusion_matrix(y_test, y_predicted_by_gaussian_naive_bayes_precission))
print('Precision, Recall, F\n', classification_report(
    y_test, y_predicted_by_gaussian_naive_bayes_precission))



Accuracy
0.8913857677902621
Confusion matrix
[[ 85  11]
 [ 18 153]]
Precision, Recall, F
               precision    recall  f1-score   support

           0       0.83      0.89      0.85        96
           1       0.93      0.89      0.91       171

    accuracy                           0.89       267
   macro avg       0.88      0.89      0.88       267
weighted avg       0.89      0.89      0.89       267



# MultinomialNB

# BernoulliNB

## Подбираем гиппер-параметры.

In [38]:
bernouli_bayes_optimal_f1 = GridSearchCV(BernoulliNB(),
                                                        {
                                                         'alpha': np.arange(0.1, 1, 1e-1),
                                                         'force_alpha': [True, False],
                                                         'fit_prior': [True, False],
                                                         },
                                                        scoring=f1_score).fit(X_train, y_train)
bernouli_bayes_optimal_precission = GridSearchCV(BernoulliNB(),
                                                        {
                                                         'alpha': np.arange(0, 1, 1e-1),
                                                         'force_alpha': [True, False],
                                                         'fit_prior': [True, False],
                                                         },
                                                        scoring=f1_score).fit(X_train, y_train)
bernouli_bayes_optimal_recall = GridSearchCV(BernoulliNB(),
                                                        {
                                                         'alpha': np.arange(0, 1, 1e-1),
                                                         'force_alpha': [True, False],
                                                         'fit_prior': [True, False],
                                                         },
                                                        scoring=f1_score).fit(X_train, y_train)


c:\Users\Mike\Desktop\ITS_project\ITS_project\lib\site-packages\sklearn\model_selection\_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Mike\Desktop\ITS_project\ITS_project\lib\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
TypeError: f1_score() takes 2 positional arguments but 3 were given

  warnings.warn(
c:\Users\Mike\Desktop\ITS_project\ITS_project\lib\site-packages\sklearn\model_selection\_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Mike\Desktop\ITS_project\ITS_project\lib\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
TypeError: f1_score() takes 2 

## Формула Байеса

In [39]:
y_predicted_by_bernouli_naive_bayes_f1 = bernouli_bayes_optimal_f1.predict(X_test)
y_predicted_by_bernouli_naive_bayes_precission = bernouli_bayes_optimal_precission.predict(X_test)
y_predicted_by_bernouli_naive_bayes_recall = bernouli_bayes_optimal_recall.predict(X_test)

## Оценка качества 

In [40]:
print('Accuracy')
print(accuracy_score(y_test, y_predicted_by_bernouli_naive_bayes_f1))
print('Confusion matrix')
print(confusion_matrix(y_test, y_predicted_by_bernouli_naive_bayes_f1))
print('Precision, Recall, F\n', classification_report(
    y_test, y_predicted_by_bernouli_naive_bayes_f1))



Accuracy
0.8539325842696629
Confusion matrix
[[ 81  15]
 [ 24 147]]
Precision, Recall, F
               precision    recall  f1-score   support

           0       0.77      0.84      0.81        96
           1       0.91      0.86      0.88       171

    accuracy                           0.85       267
   macro avg       0.84      0.85      0.84       267
weighted avg       0.86      0.85      0.86       267



In [41]:
print('Accuracy')
print(accuracy_score(y_test, y_predicted_by_bernouli_naive_bayes_precission))
print('Confusion matrix')
print(confusion_matrix(y_test, y_predicted_by_bernouli_naive_bayes_precission))
print('Precision, Recall, F\n', classification_report(
    y_test, y_predicted_by_bernouli_naive_bayes_precission))



Accuracy
0.8539325842696629
Confusion matrix
[[ 81  15]
 [ 24 147]]
Precision, Recall, F
               precision    recall  f1-score   support

           0       0.77      0.84      0.81        96
           1       0.91      0.86      0.88       171

    accuracy                           0.85       267
   macro avg       0.84      0.85      0.84       267
weighted avg       0.86      0.85      0.86       267



In [42]:
print('Accuracy')
print(accuracy_score(y_test, y_predicted_by_bernouli_naive_bayes_recall))
print('Confusion matrix')
print(confusion_matrix(y_test, y_predicted_by_bernouli_naive_bayes_recall))
print('Precision, Recall, F\n', classification_report(
    y_test, y_predicted_by_bernouli_naive_bayes_recall))



Accuracy
0.8539325842696629
Confusion matrix
[[ 81  15]
 [ 24 147]]
Precision, Recall, F
               precision    recall  f1-score   support

           0       0.77      0.84      0.81        96
           1       0.91      0.86      0.88       171

    accuracy                           0.85       267
   macro avg       0.84      0.85      0.84       267
weighted avg       0.86      0.85      0.86       267



## Подбираем гиппер-параметры.

In [27]:
np.unique(X_train+4>0)

array([ True])

In [28]:
multinominal_bayes_optimal_f1 = GridSearchCV(MultinomialNB(),
                                                        {
                                                            'alpha': np.arange(0.1, 1, 1e-1),
                                                         'force_alpha': [True, False],
                                                         'fit_prior': [True, False],
                                                         },
                                                        scoring=f1_score).fit(X_train+4, y_train)
multinominal_bayes_optimal_precission = GridSearchCV(MultinomialNB(),
                                                        {
                                                            'alpha': np.arange(0, 1, 1e-1),
                                                         'force_alpha': [True, False],
                                                         'fit_prior': [True, False],
                                                         },
                                                        scoring=f1_score).fit(X_train+4, y_train)
multinominal_bayes_optimal_recall = GridSearchCV(MultinomialNB(),
                                                        {
                                                            'alpha': np.arange(0, 1, 1e-1),
                                                         'force_alpha': [True, False],
                                                         'fit_prior': [True, False],
                                                         },
                                                        scoring=f1_score).fit(X_train+4, y_train)


c:\Users\Mike\Desktop\ITS_project\ITS_project\lib\site-packages\sklearn\model_selection\_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Mike\Desktop\ITS_project\ITS_project\lib\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
TypeError: f1_score() takes 2 positional arguments but 3 were given

  warnings.warn(
c:\Users\Mike\Desktop\ITS_project\ITS_project\lib\site-packages\sklearn\model_selection\_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Mike\Desktop\ITS_project\ITS_project\lib\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
TypeError: f1_score() takes 2 

## Формула Байеса

In [29]:
y_predicted_by_multinominal_naive_bayes_f1 = multinominal_bayes_optimal_f1.predict(X_test)
y_predicted_by_multinominal_naive_bayes_precission = multinominal_bayes_optimal_precission.predict(X_test)
y_predicted_by_multinominal_naive_bayes_recall = multinominal_bayes_optimal_recall.predict(X_test)

## Оценка качества 

In [30]:
print('Accuracy')
print(accuracy_score(y_test, y_predicted_by_multinominal_naive_bayes_f1))
print('Confusion matrix')
print(confusion_matrix(y_test, y_predicted_by_multinominal_naive_bayes_f1))
print('Precision, Recall, F\n', classification_report(
    y_test, y_predicted_by_multinominal_naive_bayes_f1))



Accuracy
0.7078651685393258
Confusion matrix
[[ 18  78]
 [  0 171]]
Precision, Recall, F
               precision    recall  f1-score   support

           0       1.00      0.19      0.32        96
           1       0.69      1.00      0.81       171

    accuracy                           0.71       267
   macro avg       0.84      0.59      0.57       267
weighted avg       0.80      0.71      0.64       267



In [32]:
print('Accuracy')
print(accuracy_score(y_test, y_predicted_by_multinominal_naive_bayes_precission))
print('Confusion matrix')
print(confusion_matrix(y_test, y_predicted_by_multinominal_naive_bayes_precission))
print('Precision, Recall, F\n', classification_report(
    y_test, y_predicted_by_multinominal_naive_bayes_precission))



Accuracy
0.7078651685393258
Confusion matrix
[[ 18  78]
 [  0 171]]
Precision, Recall, F
               precision    recall  f1-score   support

           0       1.00      0.19      0.32        96
           1       0.69      1.00      0.81       171

    accuracy                           0.71       267
   macro avg       0.84      0.59      0.57       267
weighted avg       0.80      0.71      0.64       267



In [33]:
print('Accuracy')
print(accuracy_score(y_test, y_predicted_by_multinominal_naive_bayes_recall))
print('Confusion matrix')
print(confusion_matrix(y_test, y_predicted_by_multinominal_naive_bayes_recall))
print('Precision, Recall, F\n', classification_report(
    y_test, y_predicted_by_multinominal_naive_bayes_recall))



Accuracy
0.7078651685393258
Confusion matrix
[[ 18  78]
 [  0 171]]
Precision, Recall, F
               precision    recall  f1-score   support

           0       1.00      0.19      0.32        96
           1       0.69      1.00      0.81       171

    accuracy                           0.71       267
   macro avg       0.84      0.59      0.57       267
weighted avg       0.80      0.71      0.64       267

